# Adapter Module

The `adapter` module provides bidirectional conversion between MolPy structures and external molecular libraries, primarily RDKit.

## Key Features

- Convert between RDKit `Mol` objects and MolPy `Atomistic` structures
- Generate 3D coordinates using RDKit's ETKDG method
- Optimize geometries with MMFF/UFF force fields
- Generate 2D molecular drawings (SVG)
- Stable atom mapping for round-trip conversion


In [ ]:
from rdkit import Chem

from molpy.adapter import RDKitWrapper, atomistic_to_mol, mol_to_atomistic

# Create a molecule from SMILES
mol = Chem.MolFromSmiles("CCO")

# Convert to MolPy Atomistic
atomistic = mol_to_atomistic(mol)
print(f"Converted molecule with {len(atomistic.atoms)} atoms")

# Create wrapper for bidirectional conversion
wrapper = RDKitWrapper.from_mol(mol)

# Generate 3D coordinates
wrapper.generate_3d(optimize=True)

# Access the MolPy structure
atomistic_3d = wrapper.inner
print("Generated 3D structure with coordinates")

### Converting from Atomistic to RDKit

You can also convert MolPy structures to RDKit:


In [ ]:
# Convert Atomistic back to RDKit
mol_from_atomistic = atomistic_to_mol(atomistic_3d)
print(f"RDKit molecule has {mol_from_atomistic.GetNumAtoms()} atoms")

# Generate 2D drawing
svg = wrapper.draw(show=False)
print("Generated 2D molecular drawing")